In [2]:
## import libraries
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import altair as alt

## Load data from the FL shipment cleansed files
ship_data_load_FL = pd.read_csv('/Users/sukhpreetsahota/Desktop/Duke/Fall 2022/IDS 720.01.F22/Class Project/pds-2022-yellow-team/20_intermediate_files/fl_ship_merge.csv')
ship_data_load_FL_copy = ship_data_load_FL.copy()
ship_data_load_FL_copy = ship_data_load_FL_copy[["BUYER_STATE_x", "BUYER_COUNTY_x", "YEAR", "MME", "FIPS Code", "State_x", "STNAME", "CTYNAME", "POPULATION"]]
ship_data_load_FL_copy = ship_data_load_FL_copy.rename(columns={"BUYER_STATE_x": "BUYER_STATE", "BUYER_COUNTY_x": "BUYER_COUNTY", "State_x": "State"})
ship_data_load_FL_copy['Shipment_Rate_Percentage_MME_Rate'] = ship_data_load_FL_copy['MME']/ship_data_load_FL_copy['POPULATION']
ship_data_FL = ship_data_load_FL_copy.loc[ship_data_load_FL_copy['BUYER_STATE']=='FL']
ship_data_FL_reference = ship_data_load_FL_copy.loc[ship_data_load_FL_copy['BUYER_STATE']!='FL']
ship_data_FL

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State,STNAME,CTYNAME,POPULATION,Shipment_Rate_Percentage_MME_Rate
135,FL,ALACHUA,2006,8.259662e+07,12001,Florida,Florida,ALACHUA,237199,348.216555
136,FL,ALACHUA,2007,9.525963e+07,12001,Florida,Florida,ALACHUA,240196,396.591234
137,FL,ALACHUA,2008,1.146752e+08,12001,Florida,Florida,ALACHUA,242133,473.604267
138,FL,ALACHUA,2009,1.412810e+08,12001,Florida,Florida,ALACHUA,243574,580.033370
139,FL,ALACHUA,2010,1.509108e+08,12001,Florida,Florida,ALACHUA,247624,609.435216
...,...,...,...,...,...,...,...,...,...,...
730,FL,WASHINGTON,2010,1.420824e+07,12133,Florida,Florida,WASHINGTON,24726,574.627644
731,FL,WASHINGTON,2011,1.396408e+07,12133,Florida,Florida,WASHINGTON,24516,569.590516
732,FL,WASHINGTON,2012,1.565814e+07,12133,Florida,Florida,WASHINGTON,24747,632.728645
733,FL,WASHINGTON,2013,1.621628e+07,12133,Florida,Florida,WASHINGTON,24506,661.726733


In [3]:
## Transform and Groupby MME Rate by State and Year for FL
ship_data_FL[
    "MME_Rate"
] = ship_data_FL.groupby(["BUYER_STATE", "YEAR"])[
    "Shipment_Rate_Percentage_MME_Rate"
].transform(
    "mean"
)
ship_data_FL_subset = ship_data_FL[["BUYER_STATE", "YEAR", "MME_Rate"]]
ship_data_FL_subset_grouped = ship_data_FL_subset.groupby(["BUYER_STATE", "YEAR"], as_index = False).mean()
ship_data_FL_subset_grouped_pre = ship_data_FL_subset_grouped.loc[ship_data_FL_subset_grouped["YEAR"] < 2010]
ship_data_FL_subset_grouped_post = ship_data_FL_subset_grouped.loc[ship_data_FL_subset_grouped["YEAR"] >= 2010]

/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_67001/395089536.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_data_FL[


In [4]:
## Function to create confidence interval for FL
def get_reg_fit_FL(data, yvar, xvar, alpha):
    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = alt.Chart(predictions).mark_line(color = "teal").encode(
        x=alt.X(
            xvar, 
            scale=alt.Scale(zero=False), 
            axis = alt.Axis(format="T", 
            title = "Year")), 
        y = alt.Y(
            yvar, 
            scale=alt.Scale(zero=False),
            title = "Opioid Shipments per 100,000 people in Milligrams (MME)")
    )
    ci = (
        alt.Chart(predictions)
        .mark_errorband(color = "teal")
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=""),
            y2="ci_high",
        )
    )
    chart = ci + reg
    return predictions, chart

In [32]:
## Generate Pre-Post Graphs for FL
fit, reg_chart_pre_FL = get_reg_fit_FL(
    ship_data_FL_subset_grouped_pre, 
    yvar="MME_Rate", 
    xvar="YEAR", 
    alpha=0.05
)

fit, reg_chart_post_FL = get_reg_fit_FL(
    ship_data_FL_subset_grouped_post, 
    yvar="MME_Rate", 
    xvar="YEAR", 
    alpha=0.05
)

## Create line post-policy implementation
line_2010 = alt.Chart(pd.DataFrame({'x': [2010]})).mark_rule(strokeDash=[10, 7], color = "red", strokeWidth=3).encode(x='x')

## Generate final pre-post graph for FL
pre_post_FL = reg_chart_pre_FL + reg_chart_post_FL + line_2010
pre_post_FL.properties(title="Pre-Post Shipment Rate Analysis of Florida")

alt.LayerChart(...)

In [6]:
## Include indicator for reference states for aggregation
ship_data_FL_reference["Reference_State_Indicator"] = 1
ship_data_FL_reference

/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_67001/3508536356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_data_FL_reference["Reference_State_Indicator"] = 1


,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State,STNAME,CTYNAME,POPULATION,Shipment_Rate_Percentage_MME_Rate,Reference_State_Indicator
0,AZ,APACHE,2006,4.956969e+06,4001,Arizona,Arizona,APACHE,69066,71.771475,1
1,AZ,APACHE,2007,5.611684e+06,4001,Arizona,Arizona,APACHE,69278,81.002393,1
2,AZ,APACHE,2008,5.853244e+06,4001,Arizona,Arizona,APACHE,69520,84.195107,1
3,AZ,APACHE,2009,7.344332e+06,4001,Arizona,Arizona,APACHE,70591,104.040625,1
4,AZ,APACHE,2010,7.732743e+06,4001,Arizona,Arizona,APACHE,71828,107.656394,1
...,...,...,...,...,...,...,...,...,...,...,...
1715,SC,YORK,2010,5.209517e+07,45091,South Carolina,South Carolina,YORK,226871,229.624642,1
1716,SC,YORK,2011,5.296504e+07,45091,South Carolina,South Carolina,YORK,230131,230.151692,1
1717,SC,YORK,2012,5.644417e+07,45091,South Carolina,South Carolina,YORK,234151,241.058853,1
1718,SC,YORK,2013,5.813704e+07,45091,South Carolina,South Carolina,YORK,238736,243.520190,1


In [7]:
## Transform and Groupby MME Rate by State and Year for FL Reference states
ship_data_FL_reference[
    "MME_Rate"
] = ship_data_FL_reference.groupby(["Reference_State_Indicator", "YEAR"])[
    "Shipment_Rate_Percentage_MME_Rate"
].transform(
    "mean"
)
ship_data_FL_ref_subset = ship_data_FL_reference[["YEAR", "MME_Rate"]]
ship_data_FL_ref_subset_grouped = ship_data_FL_ref_subset.groupby(["YEAR"], as_index = False).mean()
ship_data_FL_ref_subset_grouped_pre = ship_data_FL_ref_subset_grouped.loc[ship_data_FL_ref_subset_grouped["YEAR"] < 2010]
ship_data_FL_ref_subset_grouped_post = ship_data_FL_ref_subset_grouped.loc[ship_data_FL_ref_subset_grouped["YEAR"] >= 2010]

/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_67001/3961577560.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_data_FL_reference[


In [10]:
## Function to create confidence interval for FL reference states
def get_reg_fit_FL_ref(data, yvar, xvar, alpha):
    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = alt.Chart(predictions).mark_line(color = "grey", opacity=0.3).encode(
        x=alt.X(
            xvar, 
            scale=alt.Scale(zero=False), 
            axis = alt.Axis(format="T", 
            title = "Year")), 
        y = alt.Y(
            yvar, 
            scale=alt.Scale(zero=False),
            title = "Opioid Shipments per 100,000 people in Milligrams (MME)")
    )
    ci = (
        alt.Chart(predictions)
        .mark_errorband(color = "grey", opacity=0.3)
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=""),
            y2="ci_high",
        )
    )
    chart = ci + reg
    return predictions, chart

In [11]:
## Generate Pre-Post Graphs for FL reference states
fit, reg_chart_pre_FL_ref = get_reg_fit_FL_ref(
    ship_data_FL_ref_subset_grouped_pre, 
    yvar="MME_Rate", 
    xvar="YEAR", 
    alpha=0.05
)

fit, reg_chart_post_FL_ref = get_reg_fit_FL_ref(
    ship_data_FL_ref_subset_grouped_post, 
    yvar="MME_Rate", 
    xvar="YEAR", 
    alpha=0.05
)

## Create line post-policy implementation
line_2010 = alt.Chart(pd.DataFrame({'x': [2010]})).mark_rule(strokeDash=[10, 7], color = "red", strokeWidth=3).encode(x='x')

## Generate final pre-post graph for FL reference states
pre_post_FL_ref = reg_chart_pre_FL_ref + reg_chart_post_FL_ref + line_2010
pre_post_FL_ref.properties(title="Pre-Post Florida Reference States Shipment Rate Analysis")

alt.LayerChart(...)

In [12]:
## Combine pre-post graphs to create diff-in-diff graph for FL and FL reference states
diff_in_diff_FL = pre_post_FL + pre_post_FL_ref
diff_in_diff_FL.properties(title="Diff-in-Diff Shipment Rate Analysis of Florida vs Reference States")

alt.LayerChart(...)

In [13]:
## Load data from the WA shipment cleansed files
ship_data_load_WA = pd.read_csv('/Users/sukhpreetsahota/Desktop/Duke/Fall 2022/IDS 720.01.F22/Class Project/pds-2022-yellow-team/20_intermediate_files/wa_ship_merge.csv')
ship_data_load_WA_copy = ship_data_load_WA.copy()
ship_data_load_WA_copy['Shipment_Rate_Percentage_MME_Rate'] = ship_data_load_WA_copy['MME']/ship_data_load_WA_copy['POPULATION']
ship_data_WA = ship_data_load_WA_copy.loc[ship_data_load_WA_copy['BUYER_STATE']=='WA']
ship_data_WA_reference = ship_data_load_WA_copy.loc[ship_data_load_WA_copy['BUYER_STATE']!='WA']
ship_data_WA

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State,STNAME,CTYNAME,POPULATION,Shipment_Rate_Percentage_MME_Rate
1219,WA,ADAMS,2006,2.424599e+06,53001,Washington,Washington,ADAMS,16615,145.928309
1220,WA,ADAMS,2007,2.891420e+06,53001,Washington,Washington,ADAMS,16943,170.655718
1221,WA,ADAMS,2008,3.410410e+06,53001,Washington,Washington,ADAMS,17257,197.624737
1222,WA,ADAMS,2009,3.836774e+06,53001,Washington,Washington,ADAMS,17732,216.375702
1223,WA,ADAMS,2010,4.258329e+06,53001,Washington,Washington,ADAMS,18791,226.615350
...,...,...,...,...,...,...,...,...,...,...
1565,WA,YAKIMA,2010,6.746887e+07,53077,Washington,Washington,YAKIMA,244249,276.229869
1566,WA,YAKIMA,2011,7.600735e+07,53077,Washington,Washington,YAKIMA,245899,309.099890
1567,WA,YAKIMA,2012,8.023265e+07,53077,Washington,Washington,YAKIMA,246064,326.064141
1568,WA,YAKIMA,2013,8.391631e+07,53077,Washington,Washington,YAKIMA,246395,340.576342


In [14]:
## Transform and Groupby MME Rate by State and Year for WA
ship_data_WA[
    "MME_Rate"
] = ship_data_WA.groupby(["BUYER_STATE", "YEAR"])[
    "Shipment_Rate_Percentage_MME_Rate"
].transform(
    "mean"
)
ship_data_WA_subset = ship_data_WA[["BUYER_STATE", "YEAR", "MME_Rate"]]
ship_data_WA_subset_grouped = ship_data_WA_subset.groupby(["BUYER_STATE", "YEAR"], as_index = False).mean()
ship_data_WA_subset_grouped_pre = ship_data_WA_subset_grouped.loc[ship_data_WA_subset_grouped["YEAR"] < 2012]
ship_data_WA_subset_grouped_post = ship_data_WA_subset_grouped.loc[ship_data_WA_subset_grouped["YEAR"] >= 2012]

/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_67001/2976595895.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_data_WA[


In [15]:
## Function to create confidence interval for WA
def get_reg_fit_WA(data, yvar, xvar, alpha):
    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = alt.Chart(predictions).mark_line(color = "purple").encode(
        x=alt.X(
            xvar, 
            scale=alt.Scale(zero=False), 
            axis = alt.Axis(format="T", 
            title = "Year")), 
        y = alt.Y(
            yvar, 
            scale=alt.Scale(zero=False),
            title = "Opioid Shipments per 100,000 people in Milligrams (MME)")
    )
    ci = (
        alt.Chart(predictions)
        .mark_errorband(color = "purple")
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=""),
            y2="ci_high",
        )
    )
    chart = ci + reg
    return predictions, chart

In [33]:
## Generate Pre-Post Graphs for WA
fit, reg_chart_pre_WA = get_reg_fit_WA(
    ship_data_WA_subset_grouped_pre, 
    yvar="MME_Rate", 
    xvar="YEAR", 
    alpha=0.05
)

fit, reg_chart_post_WA = get_reg_fit_WA(
    ship_data_WA_subset_grouped_post, 
    yvar="MME_Rate", 
    xvar="YEAR", 
    alpha=0.05
)

## Create line post-policy implementation
line_2012 = alt.Chart(pd.DataFrame({'x': [2012]})).mark_rule(strokeDash=[10, 7], color = "red", strokeWidth=3).encode(x='x')

## Generate final pre-post graph for WA
pre_post_WA = reg_chart_pre_WA + reg_chart_post_WA + line_2012
pre_post_WA.properties(title="Pre-Post Shipment Rate Analysis of Washington")

alt.LayerChart(...)

In [17]:
## Include indicator for reference states for aggregation
ship_data_WA_reference["Reference_State_Indicator"] = 1
ship_data_WA_reference["BUYER_STATE"].unique()

/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_67001/3320543321.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_data_WA_reference["Reference_State_Indicator"] = 1


array(['AZ', 'CO', 'NY'], dtype=object)

In [18]:
## Transform and Groupby MME Rate by State and Year for WA reference states
ship_data_WA_reference[
    "MME_Rate"
] = ship_data_WA_reference.groupby(["Reference_State_Indicator", "YEAR"])[
    "Shipment_Rate_Percentage_MME_Rate"
].transform(
    "mean"
)
ship_data_WA_ref_subset = ship_data_WA_reference[["YEAR", "MME_Rate"]]
ship_data_WA_ref_subset_grouped = ship_data_WA_ref_subset.groupby(["YEAR"], as_index = False).mean()
ship_data_WA_ref_subset_grouped_pre = ship_data_WA_ref_subset_grouped.loc[ship_data_WA_ref_subset_grouped["YEAR"] < 2012]
ship_data_WA_ref_subset_grouped_post = ship_data_WA_ref_subset_grouped.loc[ship_data_WA_ref_subset_grouped["YEAR"] >= 2012]

/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_67001/922223430.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_data_WA_reference[


In [19]:
## Function to create confidence interval for WA reference states
def get_reg_fit_WA_ref(data, yvar, xvar, alpha):
    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = alt.Chart(predictions).mark_line(color = "red", opacity=0.2).encode(
        x=alt.X(
            xvar, 
            scale=alt.Scale(zero=False), 
            axis = alt.Axis(format="T", 
            title = "Year")), 
        y = alt.Y(
            yvar, 
            scale=alt.Scale(zero=False),
            title = "Opioid Shipments per 100,000 people in Milligrams (MME)")
    )
    ci = (
        alt.Chart(predictions)
        .mark_errorband(color = "red", opacity=0.2)
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=""),
            y2="ci_high",
        )
    )
    chart = ci + reg
    return predictions, chart

In [20]:
## Generate Pre-Post Graphs for WA reference states
fit, reg_chart_pre_WA_ref = get_reg_fit_WA_ref(
    ship_data_WA_ref_subset_grouped_pre, 
    yvar="MME_Rate", 
    xvar="YEAR", 
    alpha=0.05
)

fit, reg_chart_post_WA_ref = get_reg_fit_WA_ref(
    ship_data_WA_ref_subset_grouped_post, 
    yvar="MME_Rate", 
    xvar="YEAR", 
    alpha=0.05
)

## Create line post-policy implementation
line_2012 = alt.Chart(pd.DataFrame({'x': [2012]})).mark_rule(strokeDash=[10, 7], color = "red", strokeWidth=3).encode(x='x')

## Generate final pre-post graph for WA reference states
pre_post_WA_ref = reg_chart_pre_WA_ref + reg_chart_post_WA_ref + line_2012
pre_post_WA_ref.properties(title="Pre-Post Washington Reference States Shipment Rate Analysis")

alt.LayerChart(...)

In [21]:
## Combine pre-post graphs to create diff-in-diff graph for WA and WA reference states
diff_in_diff_WA = pre_post_WA + pre_post_WA_ref
diff_in_diff_WA.properties(title="Diff-in-Diff Shipment Rate Analysis of Washington vs Reference States")

alt.LayerChart(...)

In [22]:
## Load data from the TX shipment cleansed files
ship_data_load_TX = pd.read_csv('/Users/sukhpreetsahota/Desktop/Duke/Fall 2022/IDS 720.01.F22/Class Project/pds-2022-yellow-team/20_intermediate_files/tx_ship_merge.csv')
ship_data_load_TX_copy = ship_data_load_TX.copy()
ship_data_load_TX_copy = ship_data_load_TX_copy[["BUYER_STATE", "BUYER_COUNTY", "YEAR", "MONTH", "MME", "FIPS Code", "State_x", "STNAME", "CTYNAME", "POPULATION"]]
ship_data_load_TX_copy = ship_data_load_TX_copy.rename(columns={"State_x": "State"})
ship_data_load_TX_copy["YEAR_AND_MONTH_DATE"] = ship_data_load_TX_copy["YEAR"] + (
    (ship_data_load_TX_copy["MONTH"] - 1)/12)
ship_data_load_TX_copy['Shipment_Rate_Percentage_MME_Rate'] = ship_data_load_TX_copy['MME']/ship_data_load_TX_copy['POPULATION']
ship_data_TX = ship_data_load_TX_copy.loc[ship_data_load_TX_copy['BUYER_STATE']=='TX']
ship_data_TX_reference = ship_data_load_TX_copy.loc[ship_data_load_TX_copy['BUYER_STATE']!='TX']
ship_data_TX

,BUYER_STATE,BUYER_COUNTY,YEAR,MONTH,MME,FIPS Code,State,STNAME,CTYNAME,POPULATION,YEAR_AND_MONTH_DATE,Shipment_Rate_Percentage_MME_Rate
21214,TX,ANDERSON,2006,1,1.756105e+05,48001,Texas,Texas,ANDERSON,56381,2006.000000,3.114711
21215,TX,ANDERSON,2006,2,1.119858e+06,48001,Texas,Texas,ANDERSON,56381,2006.083333,19.862326
21216,TX,ANDERSON,2006,3,1.269522e+06,48001,Texas,Texas,ANDERSON,56381,2006.166667,22.516842
21217,TX,ANDERSON,2006,4,1.218623e+06,48001,Texas,Texas,ANDERSON,56381,2006.250000,21.614063
21218,TX,ANDERSON,2006,5,1.356768e+06,48001,Texas,Texas,ANDERSON,56381,2006.333333,24.064278
...,...,...,...,...,...,...,...,...,...,...,...,...
45251,TX,ZAVALA,2014,8,6.916695e+04,48507,Texas,Texas,ZAVALA,12254,2014.583333,5.644439
45252,TX,ZAVALA,2014,9,3.753480e+04,48507,Texas,Texas,ZAVALA,12254,2014.666667,3.063065
45253,TX,ZAVALA,2014,10,6.355186e+04,48507,Texas,Texas,ZAVALA,12254,2014.750000,5.186214
45254,TX,ZAVALA,2014,11,4.661580e+04,48507,Texas,Texas,ZAVALA,12254,2014.833333,3.804129


In [23]:
## Transform and Groupby MME Rate by State and Year for TX
ship_data_TX[
    "MME_Rate"
] = ship_data_TX.groupby(["BUYER_STATE", "YEAR_AND_MONTH_DATE"])[
    "Shipment_Rate_Percentage_MME_Rate"
].transform(
    "mean"
)
ship_data_TX_subset = ship_data_TX[["BUYER_STATE", "YEAR_AND_MONTH_DATE", "MME_Rate"]]
ship_data_TX_subset_grouped = ship_data_TX_subset.groupby(["BUYER_STATE", "YEAR_AND_MONTH_DATE"], as_index = False).mean()
ship_data_TX_subset_grouped_pre = ship_data_TX_subset_grouped.loc[ship_data_TX_subset_grouped["YEAR_AND_MONTH_DATE"] < 2007]
ship_data_TX_subset_grouped_post = ship_data_TX_subset_grouped.loc[ship_data_TX_subset_grouped["YEAR_AND_MONTH_DATE"] >= 2007]

/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_67001/1302209941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_data_TX[


In [24]:
## Function to create confidence interval for TX
def get_reg_fit_TX(data, yvar, xvar, alpha):
    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = alt.Chart(predictions).mark_line(color = "orange").encode(
        x=alt.X(
            xvar, 
            scale=alt.Scale(zero=False), 
            axis = alt.Axis(format="T", 
            title = "Year")), 
        y = alt.Y(
            yvar, 
            scale=alt.Scale(zero=False),
            title = "Opioid Shipments per 100,000 people in Milligrams (MME)")
    )
    ci = (
        alt.Chart(predictions)
        .mark_errorband(color = "orange")
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=""),
            y2="ci_high",
        )
    )
    chart = ci + reg
    return predictions, chart

In [26]:
## Generate Pre-Post Graphs for TX
fit, reg_chart_pre_TX = get_reg_fit_TX(
    ship_data_TX_subset_grouped_pre, 
    yvar="MME_Rate", 
    xvar="YEAR_AND_MONTH_DATE", 
    alpha=0.05
)

fit, reg_chart_post_TX = get_reg_fit_TX(
    ship_data_TX_subset_grouped_post, 
    yvar="MME_Rate", 
    xvar="YEAR_AND_MONTH_DATE", 
    alpha=0.05
)

## Create line post-policy implementation
line_2007 = alt.Chart(pd.DataFrame({'x': [2007]})).mark_rule(strokeDash=[10, 7], color = "red", strokeWidth=3).encode(x='x')

## Generate final pre-post graph for TX
pre_post_TX = reg_chart_pre_TX + reg_chart_post_TX + line_2007
pre_post_TX.properties(title="Pre-Post Shipment Rate Analysis of Texas")

alt.LayerChart(...)

In [27]:
## Include indicator for reference states for aggregation
ship_data_TX_reference["Reference_State_Indicator"] = 1
ship_data_TX_reference["BUYER_STATE"].unique()

/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_67001/217820262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_data_TX_reference["Reference_State_Indicator"] = 1


array(['IL', 'NY', 'OR'], dtype=object)

In [28]:
## Transform and Groupby MME Rate by State and Year for WA reference states
ship_data_TX_reference[
    "MME_Rate"
] = ship_data_TX_reference.groupby(["Reference_State_Indicator", "YEAR_AND_MONTH_DATE"])[
    "Shipment_Rate_Percentage_MME_Rate"
].transform(
    "mean"
)
ship_data_TX_ref_subset = ship_data_TX_reference[["YEAR_AND_MONTH_DATE", "MME_Rate"]]
ship_data_TX_ref_subset_grouped = ship_data_TX_ref_subset.groupby(["YEAR_AND_MONTH_DATE"], as_index = False).mean()
ship_data_TX_ref_subset_grouped_pre = ship_data_TX_ref_subset_grouped.loc[ship_data_TX_ref_subset_grouped["YEAR_AND_MONTH_DATE"] < 2007]
ship_data_TX_ref_subset_grouped_post = ship_data_TX_ref_subset_grouped.loc[ship_data_TX_ref_subset_grouped["YEAR_AND_MONTH_DATE"] >= 2007]

/var/folders/m7/m975ymyj3ts3w72r1_lszwjr0000gn/T/ipykernel_67001/3484320944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ship_data_TX_reference[


In [29]:
## Function to create confidence interval for WA reference states
def get_reg_fit_TX_ref(data, yvar, xvar, alpha):
    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = alt.Chart(predictions).mark_line(color = "blue", opacity=0.2).encode(
        x=alt.X(
            xvar, 
            scale=alt.Scale(zero=False), 
            axis = alt.Axis(format="T", 
            title = "Year")), 
        y = alt.Y(
            yvar, 
            scale=alt.Scale(zero=False),
            title = "Opioid Shipments per 100,000 people in Milligrams (MME)")
    )
    ci = (
        alt.Chart(predictions)
        .mark_errorband(color = "blue", opacity=0.2)
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=""),
            y2="ci_high",
        )
    )
    chart = ci + reg
    return predictions, chart

In [30]:
## Generate Pre-Post Graphs for WA reference states
fit, reg_chart_pre_TX_ref = get_reg_fit_TX_ref(
    ship_data_TX_ref_subset_grouped_pre, 
    yvar="MME_Rate", 
    xvar="YEAR_AND_MONTH_DATE", 
    alpha=0.05
)

fit, reg_chart_post_TX_ref = get_reg_fit_TX_ref(
    ship_data_TX_ref_subset_grouped_post, 
    yvar="MME_Rate", 
    xvar="YEAR_AND_MONTH_DATE", 
    alpha=0.05
)

## Create line post-policy implementation
line_2007 = alt.Chart(pd.DataFrame({'x': [2007]})).mark_rule(strokeDash=[10, 7], color = "red", strokeWidth=3).encode(x='x')

## Generate final pre-post graph for WA reference states
pre_post_TX_ref = reg_chart_pre_TX_ref + reg_chart_post_TX_ref + line_2007
pre_post_TX_ref.properties(title="Pre-Post Shipment Rate Analysis of Texas Reference States")

alt.LayerChart(...)

In [31]:
## Combine pre-post graphs to create diff-in-diff graph for WA and WA reference states
diff_in_diff_TX = pre_post_TX + pre_post_TX_ref
diff_in_diff_TX.properties(title="Diff-in-Diff Shipment Rate Analysis of Texas vs Reference States")

alt.LayerChart(...)